<h1> Segmenting and Clustering Neighborhoods in Toronto by Sabriena Tancsics </h1>

<h4 >Captsone Project Week 3: Assignment Requirements </h4>

- Start by creating a new Notebook for this assignment.
- Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,
- To create the above dataframe:
 - The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
 - Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
 - More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.
 - If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
 - Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
 - In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
- Submit a link to your Notebook on your Github repository. (10 marks)

<h4> This nootbook shall be read in sections. Each requirement is broken down into segments/ numerical order. Each bulletpoint begins with the header in the color RED </h4> 

<h2 style="color:Tomato;"> 1. Create Notebook </h2>

In [8]:
#instal beautiful soup to load HTML / WIki information
! pip install BeautifulSoup4

In [11]:
#install geopy for graphing 
!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         240 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  certifi                          2020.12.5-py36h5fab9bb_0 --> 2020.12.5-py36h5fab9bb_1



geopy-2.1.0          | 64 KB     

In [12]:
# import all & potential liabaries to be used
import pandas as pd
import numpy as np # library to handle data in a vectorized manner

import requests # library to handle requests
from bs4 import BeautifulSoup #import html file
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import os
from sklearn.cluster import KMeans # for clustering data
import folium  # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Libraries have been imported sucessfully.')

Libraries have been imported sucessfully.


<h2 style="color:Tomato;"> 2. Scrape Data from Wiki Source </h2>

In [20]:
#Link wiki/ url path 
WikiPath = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data_source = requests.get(WikiPath).text

<h2 style="color:Tomato;"> 3. Create/Complete Data Frame </h2>

In [22]:
#setting-up 'soup'
soup = BeautifulSoup(data_source, "html.parser") #"html.parser" does not require parser tools
table=soup.find('table')
column_names=['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns=column_names)

In [23]:
df #verify information is linked

,Postalcode,Borough,Neighborhood


In [26]:
#add metadata into tables

for row in table.find_all('tr'):
    row_data=[]
    for cell in row.find_all('td'):
        row_data.append(cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
        df.head()

In [27]:
df=df[df['Borough']!='Not assigned']

In [30]:
df.head(3)

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [34]:
# join data if not joined
df2=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
df2=df2.reset_index(drop=False)
df2.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [41]:
#merge postal 
dfmerge = pd.merge(df, df2, on='Postalcode')
dfmerge

,Postalcode,Borough,Neighborhood,Neighborhood_joined
0,M3A,North York,Parkwoods,Parkwoods
1,M4A,North York,Victoria Village,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights","Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","Queen's Park, Ontario Provincial Government"
...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North","The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...","Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...","Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [42]:
#drop column that no longer belongs & duplicates
dfmerge.drop(['Neighborhood'],axis=1,inplace=True)
dfmerge.drop_duplicates(inplace=True) 
dfmerge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
dfmerge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [ ]:
df3=dfmerge